In [5]:
import json
import pandas as pd
import numpy as np
import sqlite3 as sql

# Import Teams
teams = pd.read_csv('Teams.csv')

# Import Regular Season Games (basis for training data)
reg_season_df = pd.read_csv('RegularSeasonDetailedResults.csv')

In [18]:
teams = teams.set_index('Team_Id').to_dict('index')

In [19]:
reg_season_df.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lfga3,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [81]:
standard_cols = ['Season', 'season_part', 'win']
winner_cols = [col for col in reg_season_df.columns.values if col.startswith('W') and col not in ['Wloc', 'WteamName']] 
loser_cols = [col for col in reg_season_df.columns.values if col.startswith('L') and col not in ['Lloc', 'LteamName']]
game_cols = [col[1:] for col in winner_cols]

In [21]:
# Join teams so it's easier to read
reg_season_df['WteamName'] = reg_season_df['Wteam'].apply(lambda x: teams.get(x, {}).get('Team_Name'))
reg_season_df['LteamName'] = reg_season_df['Lteam'].apply(lambda x: teams.get(x, {}).get('Team_Name'))

In [31]:
# Break up regular season games into chunks
reg_season_df['season_part'] = reg_season_df['Daynum'].apply(lambda x: '1' if x <= 40 else ('2' if x <= 100 else '3'))

In [135]:
aggregated_reg_season_df = pd.DataFrame()

for team in teams:
    teams_games = reg_season_df[(reg_season_df['Wteam'] == team) | (reg_season_df['Lteam'] == team)].copy()
    teams_games['win'] = teams_games['Wteam'].apply(lambda x: 1 if x == team else 0)
    # Break out wins and losses to their own df and standardize
    wins = teams_games[teams_games['win'] == 1][standard_cols + winner_cols]
    wins.columns = standard_cols + game_cols
    losses = teams_games[teams_games['win'] == 0][standard_cols + loser_cols]
    losses.columns = standard_cols + game_cols
    teams_games = pd.concat((wins, losses), axis=0)
    stats = teams_games.groupby(['Season', 'season_part']).mean().drop('win', axis=1)
    win_count = teams_games.groupby(['Season', 'season_part'])['win'].agg(['sum', 'mean']).rename(columns={'sum': 'win_count', 'mean': 'win_percentage'})
    team_data = stats.join(win_count).reset_index()
    aggregated_reg_season_df = pd.concat((aggregated_reg_season_df, team_data))


In [139]:
aggregated_reg_season_df.sort_values(by=['win_percentage'], ascending=False)

,Season,season_part,team,score,fgm,fga,fgm3,fga3,ftm,fta,or,dr,ast,to,stl,blk,pf,win_count,win_percentage
6,2005.0,1,1243.0,67.600000,24.600000,55.000000,5.200000,12.800000,13.200000,20.000000,12.000000,28.400000,16.200000,12.600000,5.600000,2.400000,13.600000,5,1.0
2,2003.0,3,1221.0,66.875000,23.375000,51.750000,5.750000,14.750000,14.375000,21.250000,13.000000,25.125000,15.250000,12.125000,6.625000,5.250000,17.000000,8,1.0
15,2008.0,1,1450.0,69.555556,24.111111,47.222222,5.111111,15.111111,16.222222,21.111111,7.666667,24.333333,13.777778,11.777778,5.000000,3.000000,15.222222,9,1.0
12,2007.0,1,1379.0,78.000000,26.750000,55.250000,5.250000,14.750000,19.250000,34.250000,16.250000,27.750000,11.500000,17.250000,5.750000,2.250000,23.000000,4,1.0
33,2014.0,1,1455.0,75.888889,25.777778,57.222222,5.666667,17.444444,18.666667,25.555556,12.111111,28.333333,13.333333,10.111111,7.222222,4.555556,19.222222,9,1.0
17,2008.0,3,1417.0,72.300000,26.000000,57.300000,5.600000,15.500000,14.700000,21.500000,12.400000,22.400000,13.000000,10.700000,8.100000,4.500000,13.900000,10,1.0
27,2012.0,1,1462.0,74.125000,26.250000,55.875000,5.125000,13.125000,16.500000,25.750000,12.750000,28.250000,13.625000,14.500000,7.250000,4.875000,18.625000,8,1.0
6,2005.0,1,1130.0,77.000000,27.428571,59.285714,3.428571,10.857143,18.714286,25.714286,15.142857,27.285714,17.285714,14.142857,6.857143,4.571429,14.714286,7,1.0
36,2015.0,1,1449.0,70.142857,25.285714,58.142857,6.857143,20.571429,12.714286,19.571429,12.428571,28.142857,14.571429,11.142857,4.857143,7.428571,17.571429,7,1.0
8,2007.0,3,1430.0,74.166667,25.500000,49.333333,7.833333,18.333333,15.333333,21.666667,7.833333,20.333333,15.500000,9.666667,7.833333,4.500000,19.666667,6,1.0
